In [1]:
import sys
import pandas as pd
import json
import os
import pickle
import re
import difflib
from difflib import SequenceMatcher
import jellyfish
pd.set_option('display.max_columns', 100)

In [2]:
import spotipy
import spotipy.util as util

In [31]:
user_name = "Pillsbury Darboy"
client_id = "bff1afed3b63465aa6cdba9ec40f6afd"
client_secret = "53634fc86e8a4f71b7459197084e3cdf"
redirect_uri = "http://localhost/"
scope = 'user-library-read'
from spotipy.util import prompt_for_user_token
token = prompt_for_user_token(username=user_name,scope=scope,client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri)
from spotipy import Spotify
spotify = Spotify(auth=token)

In [47]:
class SetlistGenerator_Spotify(object):

    def __init__(self, seed_tracks='', seed_genres='', seed_artists=''):
        self.seed_genres = seed_genres
        self.seed_tracks = seed_tracks
        self.seed_artists = seed_artists
        
        self.AuthorizeSpotify()
        
        #Setlist starts with seed_track
        self.set_list = self.get_track_metadata_single()
        
        self.genres = ['chicago-house', 'club', 'dancehall', 'deep-house', 'detroit-techno', 'dub', 'edm',
         'electro', 'electronic', 'groove', 'house', 'idm', 'industrial', 'minimal-techno', 'progressive-house', 
          'soul', 'techno', 'trance']
        self.moods = ['happy', 'sad']
        
        self.track_pool = self.getRecommendations()
        
    def getRecommendations(self):
        # Query spotify for recommendations based on a track
        recommendations = self.spotify.recommendations(seed_artists=self.seed_artists, 
                                                       seed_genres=self.seed_genres, 
                                                       seed_tracks=self.seed_tracks, 
                                                       limit = 50)
        # Convert spotify data into readable format
        metadata = self.get_track_metadata(recommendations)
        # Retrieve audio features for spotify data
        audio_features = pd.DataFrame(self.spotify.audio_features(self.get_track_ids(recommendations)))
        df = metadata.merge(audio_features)
        
        #Correct key + mode data 
        df = self.key_mode_convert(df)
        #del df['type']
        #del df['analysis_url']
        #del df['disc_number']
        #del df['track_number']
        return(df)

    def getNextTrack(self, method = 0, min_dance=0.6):
        candidates = self.track_pool
        current_song_key = int(self.set_list[-1:]['camelot'])
        if method == 0:
            candidates = self.filter_by_camelot(df = candidates, 
                                                song_key_camelot = current_song_key)
            candidates = self.filter_by_popularity(candidates)
            candidates = self.filter_by_danceability(candidates, min_dance=min_dance)
        else:
            pass
        self.set_list = self.set_list.append(candidates.sample(1))
        self.set_list = self.set_list.reset_index(drop=True)
        
        return(self.set_list)
    
    ############################################
    ######      SEED     UTILITIES        ######
    ############################################
    
    def changeSeedTracks(self, use_last = 2):
        self.seed_tracks = list(self.set_list['id'][-use_last:])
        
    def changeSeedGenres(self, new_genres):
        self.seed_genres = new_genres
        
    def changeSeedArtists(self, new_artists):
        self.seed_genres = artists
    
    
    ############################################
    ######     TRACK   POOL   UTILITIES   ######
    ############################################
        
    def getTrackPool(self):
        return self.track_pool
    
    def expandTrackPool(self):
        self.track_pool = self.track_pool.append(self.getRecommendations())
        self.cleanTrackPool()
        
        print('Expanded track pool')
    
    def cleanTrackPool(self):
        self.track_pool = self.track_pool[-self.track_pool['id'].isin(list(self.set_list['id']))]
        self.track_pool = self.track_pool.drop_duplicates()
        self.track_pool = self.track_pool.reset_index(drop=True)

        
    ############################################
    ######    TRACK SELECTION UTILITIES   ######
    ############################################
        
    def closest_keys(self, camelot):
        closest = []
        closest.append(camelot)
        closest.append((camelot + 1)%12)
        closest.append((camelot - 1)%12)
        return(closest)
    
    def filter_by_camelot(self, df, song_key_camelot):
        filtered = []
        for i in self.closest_keys(song_key_camelot):
            filtered.append(pd.DataFrame(df[df['camelot'] == i]))
        return(pd.concat(filtered))
        
    def filter_by_popularity(self, df, min_pop=0, max_pop=100):
        df = df[df['popularity'] >= min_pop]
        df = df[df['popularity'] <= max_pop]
        return(df)
    
    def filter_by_tempo(self, df, min_tempo = 90, max_tempo = 145):
        df = df[df['tempo'] >= min_tempo]
        df = df[df['tempo'] <= max_tempo]
        return(df)
    
    def filter_by_valence(self, df=None, min_valence = 0, max_valence = 1):
        df = self.track_pool
        df = df[df['valence'] >= min_valence]
        df = df[df['valence'] <= max_valence]
        return(df)
    
    def filter_by_danceability(self, df=None, min_dance = 0, max_dance = 1):
        df = self.track_pool
        df = df[df['danceability'] >= min_dance]
        df = df[df['danceability'] <= max_dance]
        return(df)
    
    def filter_by_energy(self, df=None, min_energy = 0, max_energy = 1):
        df = self.track_pool
        df = df[df['energy'] >= min_energy]
        df = df[df['energy'] <= max_energy]
        return(df)



    ############################################
    ######        METADATA UTILITIES      ######
    ############################################
    
    def get_track_metadata(self, recommendations):
        tracklist_frames = []
        for track in recommendations['tracks']:
            tracklist_frames.append({'track': track['name'],'artist': track['artists'][0]['name'],
                                     'duration':track['duration_ms'],
                                    'track_number': track['track_number'], 'id': track['id'], 
                                     'popularity':track['popularity'],
                                     'disc_number':track['disc_number'], 'album':track['album']['name']})
        return(pd.DataFrame(tracklist_frames))

    def get_track_metadata_single(self):
        track_id = self.seed_tracks[0]
        track = self.spotify.track(track_id)
        metadata = pd.DataFrame({'track': track['name'],'artist': track['artists'][0]['name'],
                                     'duration':track['duration_ms'],
                                    'track_number': track['track_number'], 'id': track['id'], 
                                     'popularity':track['popularity'],
                                     'disc_number':track['disc_number'], 'album':track['album']['name']}, index =[0])
        audio_features = pd.DataFrame(self.spotify.audio_features(track_id))
        df = metadata.merge(audio_features)
        df = self.key_mode_convert(df)
        return(df)
    
    def get_track_ids(self, recommendations):
        id_list = []
        for spotify_id in recommendations['tracks']:
            id_list.append(spotify_id['id'])
        return(id_list)
    
    def key_mode_convert(self, recommendations):
        df = recommendations
        camelot_list = []
        for i in range(0, len(df)):
            if df['mode'][i] == 1:
                if (df['key'][i] == 0):
                    camelot_list.append(7)
                if (df['key'][i] == 1):
                    camelot_list.append(2)
                if (df['key'][i] == 2):
                    camelot_list.append(9)
                if (df['key'][i] == 3):
                    camelot_list.append(4)
                if (df['key'][i] == 4):
                    camelot_list.append(11)                
                if (df['key'][i] == 5):
                    camelot_list.append(6)                
                if (df['key'][i] == 6):
                    camelot_list.append(1)                
                if (df['key'][i] == 7):
                    camelot_list.append(8)                
                if (df['key'][i] == 8):
                    camelot_list.append(3)                
                if (df['key'][i] == 9):
                    camelot_list.append(10)                
                if (df['key'][i] == 10):
                    camelot_list.append(5)
                if (df['key'][i] == 11):
                    camelot_list.append(0)
            else:
                if (df['key'][i] == 0):
                    camelot_list.append(4)
                if (df['key'][i] == 1):
                    camelot_list.append(11)
                if (df['key'][i] == 2):
                    camelot_list.append(6)
                if (df['key'][i] == 3):
                    camelot_list.append(1)
                if (df['key'][i] == 4):
                    camelot_list.append(8)                
                if (df['key'][i] == 5):
                    camelot_list.append(3)                
                if (df['key'][i] == 6):
                    camelot_list.append(10)                
                if (df['key'][i] == 7):
                    camelot_list.append(5)                
                if (df['key'][i] == 8):
                    camelot_list.append(0)                
                if (df['key'][i] == 9):
                    camelot_list.append(7)                
                if (df['key'][i] == 10):
                    camelot_list.append(2)
                if (df['key'][i] == 11):
                    camelot_list.append(9)
        df['camelot'] = camelot_list
        return(df)

    ############################################
    ######        SPOTIFY  UTILITIES      ######
    ############################################
    
    def SearchSpotify(self, search_string):
        return self.spotify.search(search_string)
    

    
    def AuthorizeSpotify(self):
        print('Authorizing Spotify')
        self.user_name = "Ronald Marp"
        self.client_id = '1e05e910ef664f27949005267fdd0936'
        self.client_secret ='7e65fb89a86a4a9c849924f5bd95661a'
        self.redirect_uri = "http://localhost/"
        self.scope = 'user-library-read'
        self.token = prompt_for_user_token(username=self.user_name,scope=self.scope,
                                           client_id=self.client_id,
                                           client_secret=self.client_secret,
                                           redirect_uri=self.redirect_uri)
        self.spotify = Spotify(auth=self.token)

In [20]:
## Percolator Meme - Teeth ['2QT5afoBT6nQabNuvcxXUa']

In [48]:
def search_for_artistID(search_string):
    result = spotify.search(str(search_string))['tracks']['items'][0]['artists'][0]
    print('Found artist %s' %result['name'])
    return(result['id'])

In [49]:
def search_for_trackID(search_string):
    result = spotify.search(str(search_string))['tracks']['items'][0]
    print('Found track %s by %s' %(result['name'], result['artists'][0]['name']))
    return(result['id'])

In [50]:
seed_artist_id = search_for_artistID('Aleksi Perala')

SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/search?limit=10&q=Aleksi+Perala&type=track&offset=0:
 The access token expired

In [51]:
seed_track_id = search_for_trackID('Donkey Rhubarb Aphex Twin')

SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/search?limit=10&q=Donkey+Rhubarb+Aphex+Twin&type=track&offset=0:
 The access token expired

In [52]:
d = SetlistGenerator_Spotify(seed_tracks=[seed_track_id], 
                             #seed_artists=[seed_artist_id], 
                             seed_genres=['minimal-techno', 'techno']) #seed_genres=['minimal-techno', 'techno', 'detroit-techno'])

Authorizing Spotify


            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?redirect_uri=http%3A%2F%2Flocalhost%2F&response_type=code&client_id=1e05e910ef664f27949005267fdd0936&scope=user-library-read in your browser


Enter the URL you were redirected to: http://localhost/?code=AQCg7fioDXG_3fTBjyw-MMmC_VvYNUYLxeBlcrMVcwk7mXnCzISui664k-zhn3OiavQTiGqB4E814QMHcma6_igJKocSfYFKspDMJGUCI7iCuvZpUBny5tB9_D6NQZYksYM7mxyGhWu4fod6A8Dad5BFQEafBGtP9HijkL6DR2jp08lbNkDNKYQRTjBHXPY-r-k9sY2HRL8qvA




In [42]:

while len(d.set_list) < 10:
    d.getNextTrack()
    d.changeSeedTracks()
    d.expandTrackPool()

d.set_list

Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool


,album,artist,disc_number,duration,id,popularity,track,track_number,acousticness,analysis_url,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence,camelot
0,Donkey Rhubarb,Aphex Twin,1,368173,0c0PhGTdl4K1SRWu0kYkBh,26,Donkey Rhubarb,1,0.000555,https://api.spotify.com/v1/audio-analysis/0c0P...,0.745,368173,0.890,0.926,0,0.2030,-8.510,1,0.0685,140.017,4,https://api.spotify.com/v1/tracks/0c0PhGTdl4K1...,audio_features,spotify:track:0c0PhGTdl4K1SRWu0kYkBh,0.714,7
1,The Collective,Octave One,1,342284,7ADwofH1sfYLkWOdRsqYzc,6,Eniac - Original Mix,3,0.000005,https://api.spotify.com/v1/audio-analysis/7ADw...,0.779,342284,0.805,0.908,3,0.1020,-11.012,0,0.0404,130.598,4,https://api.spotify.com/v1/tracks/7ADwofH1sfYL...,audio_features,spotify:track:7ADwofH1sfYLkWOdRsqYzc,0.796,1
2,Devil's Advocate,Dave Clarke,1,284000,3o3FV1l7mxr7jUPAhjCxpV,0,What Was Her Name,2,0.001440,https://api.spotify.com/v1/audio-analysis/3o3F...,0.640,284000,0.515,0.468,8,0.0935,-11.920,1,0.0607,144.279,4,https://api.spotify.com/v1/tracks/3o3FV1l7mxr7...,audio_features,spotify:track:3o3FV1l7mxr7jUPAhjCxpV,0.411,3
3,Binky's Groove EP,Mr. G,1,361803,1XkWhPTmlyJNIl6Ylq0AmH,6,Dis-tracted!,4,0.000923,https://api.spotify.com/v1/audio-analysis/1XkW...,0.774,361803,0.833,0.853,0,0.0649,-8.731,1,0.0709,127.984,4,https://api.spotify.com/v1/tracks/1XkWhPTmlyJN...,audio_features,spotify:track:1XkWhPTmlyJNIl6Ylq0AmH,0.155,7
4,Human Reason (Len Faki Remix) / Memesis,Adam Beyer,1,480424,3fNPyF3qbn4ZJL3yXmxsfx,18,Memesis,2,0.026000,https://api.spotify.com/v1/audio-analysis/3fNP...,0.793,480425,0.888,0.856,4,0.0856,-7.103,1,0.2120,126.000,4,https://api.spotify.com/v1/tracks/3fNPyF3qbn4Z...,audio_features,spotify:track:3fNPyF3qbn4ZJL3yXmxsfx,0.481,11
5,Voyager EP,DJ 3000,1,410078,7GvchCl4JbuJeP0WWpK7ek,3,Saz,2,0.003130,https://api.spotify.com/v1/audio-analysis/7Gvc...,0.800,410079,0.643,0.879,6,0.0794,-10.400,1,0.0491,127.007,4,https://api.spotify.com/v1/tracks/7GvchCl4JbuJ...,audio_features,spotify:track:7GvchCl4JbuJeP0WWpK7ek,0.890,1
6,Moodymann,Moodymann,1,343000,4ncFRft2xEs4kanULQjaOz,47,Lyk U Use 2 (feat. Andres),6,0.108000,https://api.spotify.com/v1/audio-analysis/4ncF...,0.621,343000,0.485,0.130,11,0.1560,-13.429,0,0.2190,183.391,4,https://api.spotify.com/v1/tracks/4ncFRft2xEs4...,audio_features,spotify:track:4ncFRft2xEs4kanULQjaOz,0.140,9
7,Here Today Gone Tomorrow,Fritz Kalkbrenner,1,357634,7c9kPb3onWgaPDaL0KOkci,1,Kings In Exile,2,0.072100,https://api.spotify.com/v1/audio-analysis/7c9k...,0.738,357634,0.734,0.748,4,0.0394,-11.109,1,0.0374,125.990,4,https://api.spotify.com/v1/tracks/7c9kPb3onWga...,audio_features,spotify:track:7c9kPb3onWgaPDaL0KOkci,0.389,11
8,Lost Memory,Boris Brejcha,1,462719,76lT30VRv09h5MQp5snmsb,51,Lost Memory,1,0.014600,https://api.spotify.com/v1/audio-analysis/76lT...,0.876,462720,0.458,0.908,11,0.1110,-11.519,1,0.0763,124.999,3,https://api.spotify.com/v1/tracks/76lT30VRv09h...,audio_features,spotify:track:76lT30VRv09h5MQp5snmsb,0.139,0
9,Logic Memory Center,John Tejada,1,295891,7vjdc9fZiG4ckMA140NB9I,14,Unit B1656,2,0.000102,https://api.spotify.com/v1/audio-analysis/7vjd...,0.800,295892,0.706,0.853,10,0.0486,-9.182,0,0.0633,125.037,4,https://api.spotify.com/v1/tracks/7vjdc9fZiG4c...,audio_features,spotify:track:7vjdc9fZiG4ckMA140NB9I,0.459,2


In [39]:
d.set_list['uri'].to_csv('../../../../spotify-downloader/output.txt', index=False)

In [ ]:
len(mixesdb)

In [ ]:
# Load and Process Datasets for big-data generation

spotify = pd.read_csv('data/spotify/spotify_database_mergedV2.csv')
del spotify['Unnamed: 0']
del spotify['Unnamed: 0.1']
spotify['artist_and_track'] = spotify['artist'] + ' ' + spotify['track']
print("Read and processed Spotify data")

discogs_m = pd.read_csv('data/discogs/discogs_mixed.csv')
discogs_r = pd.read_csv('data/discogs/discogs_releases.csv')
to_concat = [discogs_m, discogs_r]
discogs = pd.concat(to_concat)
del discogs_m
del discogs_r
del discogs['Unnamed: 0']
del discogs['genre']
del discogs['labels']
del discogs['length']
discogs['artist'] = discogs['artist'].apply(lambda x: re.sub(pattern='\([0-9]*\)', string=x, repl = '').strip())
discogs['artist_and_track'] = discogs['artist'] + ' ' + discogs['track']
print("Read and processed Discogs data")

mixesdb = pd.read_csv('data/mixesdb/mixesdb_all_sets_featurized.csv')
del mixesdb['Unnamed: 0']
mixesdb['artist_and_track'] = mixesdb['artist'] + ' ' + mixesdb['track']
print("Read and processed MixesDB data")


In [ ]:
spotify['popularity'].max()